In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !cp /content/gdrive/MyDrive/STT_project/data_train.tar.xz '/content/'
# !cp /content/gdrive/MyDrive/STT_project/data_dev.tar.xz '/content/'

In [ ]:
# !tar -xf /content/data_train.tar.xz
# !tar -xf /content/data_dev.tar.xz

In [ ]:
# install dependencies

!pip install pytorch-lightning
!pip install torchdata
!pip install boto3
!pip install gradio

In [ ]:
import copy
import gradio as gr
import math
import pandas as pd
import numpy as np
import os
import re
import requests
import torch
import torch.nn.functional as F
import torchaudio
from bs4 import BeautifulSoup
from pytorch_lightning import LightningModule, LightningDataModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything
from sklearn.model_selection import train_test_split
from torchmetrics.functional import char_error_rate, word_error_rate
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
# instatiate wav2vec2 asr and pretrained object
BUNDLE_ASR = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
BUNDLE_PRE = torchaudio.pipelines.WAV2VEC2_BASE

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.__version__)
print(torchaudio.__version__)
print(DEVICE)

1.13.1+cu116
0.13.1+cu116
cuda


## Scrape QA data

In [ ]:
url_list = [
    'https://www.simplilearn.com/tutorials/data-science-tutorial/data-science-interview-questions',
    'https://intellipaat.com/blog/interview-question/data-science-interview-questions/',
    'https://www.edureka.co/blog/interview-questions/data-science-interview-questions/',
    'https://www.interviewbit.com/data-science-interview-questions/',
    'https://hackr.io/blog/data-science-interview-questions',
    'https://www.springboard.com/blog/data-science/data-science-interview-questions/',
    'https://www.interviewkickstart.com/interview-questions/data-scientist-interview-questions',
    'https://www.projectpro.io/article/100-data-science-interview-questions-and-answers-for-2021/184',
    'https://towardsdatascience.com/over-100-data-scientist-interview-questions-and-answers-c5a66186769a',
    'https://brainstation.io/career-guides/data-science-interview-questions',
    'https://www.indeed.com/career-advice/interviewing/data-science-interview-questions',
    'https://mindmajix.com/data-science-interview-questions',
    'https://www.analyticsvidhya.com/blog/2021/04/20-data-science-interview-questions-for-a-beginner/',
    'https://www.v7labs.com/blog/data-science-interview-questions-and-answers',
    'https://ai.plainenglish.io/data-science-interview-questions-part-2-regression-analysis-fd3406e90b1a',
    'https://www.javatpoint.com/data-science-interview-questions',
    'https://blog.imocha.io/data-science-interview-questions-and-answers',
    'https://firsthand.co/blogs/interviewing/6-common-data-science-interview-questions',
    'https://360digitmg.com/top-data-science-interview-questions-answers',
    'https://iq.opengenus.org/advanced-interview-questions-on-data-science/',
    'https://quanthub.com/data-science-interview-questions/',
    'https://skill-lync.com/blogs/top-8-most-frequently-asked-questions-in-data-science-job-interviews',
    'https://www.bitdegree.org/tutorials/data-science-interview-questions/',
    'https://www.testpreptraining.com/blog/top-70-microsoft-data-science-interview-questions/',
    'https://machinelearninggeek.com/data-science-interview-questions-part-6-nlp-text-mining/',
    'https://www.hackertrail.com/talent/backend/data-science-interview-questions/',
    'https://nitin-panwar.github.io/Top-100-Data-science-interview-questions/'
]

In [ ]:
question_tokens = ['Q',
                  'What',
                  'Explain',
                  'If ',
                  'How',
                  'Which',
                  'Why',
                  'Where',
                  'When',
                  'Differentiate',
                  'In ',
                  'You ',
                  'For ',
                  'Do ',
                  'Describe',
                  'Difference',
                  'Given',
                  'Based',
                  'We ',
                  'Mention',
                  'Out ',
                  'However',
                  'Walkthrough',
                  'A ',
                  'Can ',
                  'Are ',
                  'Define',
                  'Suppose',
                  'During',
                  'Give',
                  'Is ',
                  'Between',
                  'State',
                  'Please',
                  'Supervised',
                  'K-Means',
                  'Quota',
                  'Find',
                  'Quickly',
                  'Use',
                  'Assume',
                  'Make',
                  'Provide',
                  'Consider',
                  'An ',
                  'To ',
                  'List',
                  'can',
                  'Name',
                  'Variance',
                  'Calculate',
                  'WHAT']

In [ ]:
def scrape_data():
    """
    A function to crawl a list of websites, extract question answer pairs.
    Returns dataframe of question and answer columns.
    """

    ds_dict = {}
    for count, url in enumerate(tqdm(url_list)):
        response = requests.get(url).text
        soup = BeautifulSoup(response, "html.parser")
        for tag in soup.find_all():
            if len(tag.text) > 10:
                if tag.text.endswith(('?', '? ', '?\n', '?\n\n')) or \
                 tag.text[:1].isdigit() and tag.text[1] == '.' or \
                 tag.text.startswith(tuple(question_tokens)):
                    if tag.find_next('p'):
                        if tag.text in ds_dict:
                            tag_text = f'dup_{count}_{tag.text}'
                        else:
                            tag_text = tag.text

                        if tag.find_next('p').text not in ds_dict.values():
                            ds_dict[tag_text] = tag.find_next('p').text

    df = pd.DataFrame(list(ds_dict.items()), columns = ['Question','Answer'])
    df.to_csv('QA.csv')

In [ ]:
# data_csv = '/content/QA - QA.csv'
# df = pd.read_csv(data_csv, names=['Questions', 'Answers'])
# df = df.replace('\n','', regex=True)
# df

In [ ]:
# questions = df["Questions"].tolist()
# max_len = max([len(question) for question in questions])
# min_len = min([len(question) for question in questions])
# print(max_len)
# print(min_len)

# Preprocess


In [ ]:
DATASET_TRAIN = "/content/data_train"

In [ ]:
def get_transcript(transcript_file):
    """Loads transcript file. Extracts and returns audio transcription."""

    with open(transcript_file, "r", encoding='utf-8') as f:
        transcript_line = f.readline().strip()
        transcript = re.split('(?<=.){(?=.*)', transcript_line, maxsplit=1, flags=re.IGNORECASE)[0]
    return transcript


def get_transcript_audio_pair_list(dataset_dir):
    """
    Takes directory of audio and transcript files.
    Returns numpy array of audio paths and corresponding transcriptions.
    """

    wavs = set()
    transcripts = set()
    wav_path = ''
    transcript_path = ''
    for root, dirs, files in os.walk(dataset_dir):
        if files:
            for f in files:
                if f.endswith('.wav'):
                    wavs.add(f[:-4])
                    if not wav_path:
                        wav_path = root
                if f.endswith('.txt'):
                    transcripts.add(f[:-4])
                    if not transcript_path:
                        transcript_path = root

    pair_list = [(f"{wav_path}/{fname}.wav", get_transcript(f"{transcript_path}/{fname}.txt")) for fname in wavs if fname in transcripts]
    return np.asarray(pair_list)

In [ ]:
data = get_transcript_audio_pair_list(DATASET_TRAIN)
print(data[:5])

[['/content/data_train/clips/8000_2_VK_16k_0034.wav'
  'Describe Cross-validation Describe Markov chains?']
 ['/content/data_train/clips/8000_7_SC_16k_mono_0389.wav'
  'Explain What is meant by Deep Learning?']
 ['/content/data_train/clips/8000_1_RP_16k_0111.wav'
  'How do you extract features in NLP?']
 ['/content/data_train/clips/8000_5_SB_16k_0075.wav'
  'Explain Unsupervised Clustering approach?']
 ['/content/data_train/clips/8000_2_VK_16k_0581.wav'
  'What is the Sampling Frame in the SRS sampling technique?']]


## Prepare

In [ ]:
# get pretrain model sample rate
SAMPLE_RATE = BUNDLE_PRE.sample_rate
# load and sort labels
LABELS = BUNDLE_ASR.get_labels()
LABELS = LABELS + ('[UNK]',)
LABELS = sorted(LABELS)
# here '-' is arrenged to be at index 0 and corresponds to CTC pad token
LABELS = ['-']+[elem for elem in LABELS if elem != '-']

In [ ]:
class InputProcessor:
    """Class for preprocessing text and audio for training and inference."""

    def __init__(self, text=False, audio=False, inference=False, labels=[], sample_rate=int()):
        """Args:
            text (bool, optional): Raw text transcriptions. Outputs will be preprocessed text.
            audio (bool, optional): Raw audio paths. Outputs will be preprcessed audio.
            inference (bool, optional): If True, the output of the class will be
            retricted to specific preprocesses.
            labels (:obj: list, optional): List of output labels (vocabulary).
            sample_rate (int, optional): Integer value sample rate of audio model was trained with.
        """

        self.text = text
        self.audio = audio
        self.inference = inference

        if self.text and not self.inference:
            # create character to index mapping dict. Index used to vectorize transcript.
            self.label_to_id_mapping = {char: i for i, char in enumerate(labels)}
            # will be used in place of whitespace.
            self.delimiter_label = "|"
        if self.audio:
            self.sample_rate = sample_rate

    def _clean_transcript(self, transcript):
        """Clean transcript of all characters except those found in the labels list."""

        transcript = transcript.replace("-", " ").replace("/", " ")
        transcript = re.sub("[^A-Z' ]", "", transcript.upper())
        return transcript

    def _tokenize_transcript(self, transcript):
        """Convert alpha characters to numeric values. Used as output during training."""

        transcript_token_list = [self.label_to_id_mapping[c] for c in transcript.replace(" ", self.delimiter_label)]
        return transcript_token_list

    def _resample_waveform(self, waveform):
        """Converts sample rate to that of audio trained in pretrained model."""

        waveform, sample_rate = torchaudio.load(waveform)
        if sample_rate != self.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sample_rate, self.sample_rate)
        return waveform

    def _normalize_audio(self, waveform):
        """Mean and standard deviation normalization.
        Ensures model isn't biased too much to speaker specific features."""

        waveform_mean = torch.mean(waveform)
        waveform_std = torch.std(waveform)
        return (waveform - waveform_mean) / (waveform_std + 1e-10)

    def __call__(self, input_data):
        """To apply class preprocessing methods functionally to input text and audio."""

        if self.text:
            transcript = self._clean_transcript(input_data)
            if not self.inference:
                transcript = transcript + self.delimiter_label
                transcript = self._tokenize_transcript(transcript)
            return transcript
        elif self.audio:
            waveform = self._resample_waveform(input_data)
            waveform = self._normalize_audio(waveform)
            if not self.inference:
                waveform_length = waveform.size(1)
                return waveform, waveform_length
            return waveform


class PPDataset(Dataset):
    """Dataset object to load and process training and eval data."""

    def __init__(self, input_data, labels, sample_rate):
        """Args:
            input_data (:obj: `list`): Array of audio, transcript arrays.
            labels (:obj: list, optional): List of output labels (vocabulary).
            sample_rate (int, optional): Integer value sample rate of audio model was trained with.
        """

        self.tokenizer = InputProcessor(text=True, labels=labels)
        self.extractor = InputProcessor(audio=True, sample_rate=sample_rate)
        self.input_data = input_data

    def __len__(self):
        """Return len of input_data. """

        return len(self.input_data)

    def __getitem__(self, idx):
        """
        Preprocesses batch data at index in array of audio, transcript pairs.
        Returns set of tensors containing numeric representations of training data.
        """

        audio, text = self.input_data[idx]
        waveform, waveform_length = self.extractor(audio)
        label = torch.tensor(self.tokenizer(text))
        label_length = label.size(0)
        return (waveform, waveform_length, label, label_length)

In [ ]:
ds = PPDataset(data[:5], LABELS, SAMPLE_RATE)
for item in ds:
    print(item)

(tensor([[-0.0697, -0.0819, -0.0682,  ..., -0.1168, -0.0864, -0.0819]]), 155195, tensor([ 5,  6, 20,  4, 19, 10,  3,  6, 29,  4, 19, 16, 20, 20, 29, 23,  2, 13,
        10,  5,  2, 21, 10, 16, 15, 29,  5,  6, 20,  4, 19, 10,  3,  6, 29, 14,
         2, 19, 12, 16, 23, 29,  4,  9,  2, 10, 15, 20, 29]), 49)
(tensor([[ 0.1269,  0.1214,  0.1338,  ..., -0.6436, -0.6629, -0.6670]]), 76158, tensor([ 6, 25, 17, 13,  2, 10, 15, 29, 24,  9,  2, 21, 29, 10, 20, 29, 14,  6,
         2, 15, 21, 29,  3, 26, 29,  5,  6,  6, 17, 29, 13,  6,  2, 19, 15, 10,
        15,  8, 29]), 39)
(tensor([[ 0.0020,  0.0020,  0.0020,  ...,  0.0004, -0.0012,  0.0004]]), 65598, tensor([ 9, 16, 24, 29,  5, 16, 29, 26, 16, 22, 29,  6, 25, 21, 19,  2,  4, 21,
        29,  7,  6,  2, 21, 22, 19,  6, 20, 29, 10, 15, 29, 15, 13, 17, 29]), 35)
(tensor([[-0.0257, -0.0264, -0.0312,  ...,  0.0304,  0.0332,  0.0373]]), 65598, tensor([ 6, 25, 17, 13,  2, 10, 15, 29, 22, 15, 20, 22, 17,  6, 19, 23, 10, 20,
         6,  5, 29,  4, 1

In [ ]:
class CollateFn:
    """A custom collate class to process variable length input and ouput batch data."""

    def pad_data(self, batch, waveforms, labels):
        """Pads input and output data to length of lengest input/output."""

        waveform_sizes = [sample[0].shape[1] for sample in batch]
        waveform_size = max(waveform_sizes)
        waveforms_padded = torch.zeros(len(batch), waveform_size)
        for i in range(len(waveforms)):
            waveforms_padded[i][0:waveforms[i].shape[1]] = waveforms[i]
        labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-1)
        return waveforms_padded, labels_padded

    def __call__(self, batch):
        """Applies collate processes functionally to batch data"""

        waveforms = []
        wav_lens = []
        labels = []
        lab_lens = []
        for sample in batch:
            waveforms.append(sample[0])
            wav_lens.append(sample[1])
            labels.append(sample[2])
            lab_lens.append(sample[3])

        wavs_padded, labs_padded = self.pad_data(batch, waveforms, labels)
        wav_lens_tensor, lab_lens_tensor = torch.tensor(wav_lens), torch.tensor(lab_lens)
        return wavs_padded, wav_lens_tensor.int(), labs_padded.int(), lab_lens_tensor.int()


class DataModule(LightningDataModule):
    """Custom Lightning Data Module class. Prepares and loads training can validation data."""

    def __init__(self, data, labels, sample_rate, batch_size):
        """Args:
            data (:obj: `list`): Numpy array of audio, transcript arrays.
            labels (:obj: list, optional): List of output labels (vocabulary).
            sample_rate (int, optional): Integer value sample rate of audio model was trained with.
            batch_size(int): Batch size value.
        """

        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.labels = labels
        self.sample_rate = sample_rate

    def setup(self, stage=None):
        """Randomized split of train and validation data. Returns sorted splits."""

        self.train_data, self.val_data = train_test_split(self.data,
                                                          test_size=0.2,
                                                          shuffle=True)

        # Sort split by output length to ensure similar length datapoints grouped together.
        # This is to avoid overpadding of variable length outputs.
        self.train_data = sorted(self.train_data, key = lambda x: x[1])
        self.val_data = sorted(self.val_data, key = lambda x: x[1])

    def train_dataloader(self):
        """Loads traning data."""

        return DataLoader(dataset=PPDataset(self.train_data, self.labels, self.sample_rate),
                          batch_size=self.batch_size,
                          drop_last=False,
                          collate_fn=CollateFn(),
                          num_workers=0,
                          shuffle=False)

    def val_dataloader(self):
        """Loads validation data."""

        return DataLoader(dataset=PPDataset(self.val_data, self.labels, self.sample_rate),
                          batch_size=self.batch_size,
                          drop_last=False,
                          collate_fn=CollateFn(),
                          num_workers=0)

In [ ]:
dm = DataModule(data[:5], LABELS, SAMPLE_RATE, batch_size=4)
dm.setup()

for batch in dm.train_dataloader():
    print(batch)

(tensor([[-0.0697, -0.0819, -0.0682,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1269,  0.1214,  0.1338,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0020,  0.0020,  0.0020,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0058,  0.0116,  0.0443,  ...,  0.0827,  0.0847,  0.0462]]), tensor([155195,  76158,  65598, 155515], dtype=torch.int32), tensor([[ 5,  6, 20,  4, 19, 10,  3,  6, 29,  4, 19, 16, 20, 20, 29, 23,  2, 13,
         10,  5,  2, 21, 10, 16, 15, 29,  5,  6, 20,  4, 19, 10,  3,  6, 29, 14,
          2, 19, 12, 16, 23, 29,  4,  9,  2, 10, 15, 20, 29, -1, -1, -1, -1, -1,
         -1, -1, -1],
        [ 6, 25, 17, 13,  2, 10, 15, 29, 24,  9,  2, 21, 29, 10, 20, 29, 14,  6,
          2, 15, 21, 29,  3, 26, 29,  5,  6,  6, 17, 29, 13,  6,  2, 19, 15, 10,
         15,  8, 29, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1],
        [ 9, 16, 24, 29,  5, 16, 29, 26, 16, 22, 29,  6, 25, 21, 19,  2,  4, 21,
         29,  7,  6,  2, 21, 22, 19,  6, 20, 

## Model

In [ ]:
def get_transcription(transcription, labels):
    """Converts vocabulary index to character and returns transcription."""

    # create dict of vocabulary index, character pairs.
    map_dict = {i: char.lower() for i, char in enumerate(labels)}
    transcription = "".join(map_dict[int(i)] for i in transcription if i != int('-1')).replace("|", " ").strip()
    return transcription


def viterbi_decode(emission, labels, blank_idx=0):
    """
    Given a sequence emission over labels, get the highest probability sequence path.
    Returns hypothesis transcription.
    """

    hypothesis = emission.argmax(-1).unique_consecutive()
    hypothesis = hypothesis[hypothesis != blank_idx]
    hypothesis = get_transcription(hypothesis, labels)
    return hypothesis


def get_error_rates(preds, targets):
    """Given list of corresponding transcription returns character and word error rates."""

    wer = word_error_rate(preds=preds, target=targets)
    cer = char_error_rate(preds=preds, target=targets)
    return cer, wer

In [ ]:
class BiStageLRScheduler(torch.optim.lr_scheduler._LRScheduler):
    """Custom shceduler class for two stage learning rate scheduling."""

    def __init__(self, optimizer, warmup_updates, decay_updates):
        """Args:
            optimizer (:obj:): Numpy array of audio, transcript arrays.
            warmup_updates (int): number of steps that warmup updates applied.
            decay_updates (int): number of steps that decay updates applied.
        """

        self.warmup_updates = warmup_updates
        self.decay_updates = decay_updates
        # scale multipliers
        self.init_lr_scale = 0.01
        self.final_lr_scale = 0.05
        super().__init__(optimizer, last_epoch=-1)

    def get_lr(self):
        """reduces learing rate by calculated factors. Returns adjust learning rate."""

        base_lrs_out = []
        for base_lr in self.base_lrs:
            if self._step_count <= self.warmup_updates:
                base_lr = base_lr * (self.init_lr_scale + self._step_count / self.warmup_updates * (1 - self.init_lr_scale))
            elif self._step_count <= (self.warmup_updates + self.decay_updates):
                base_lr = base_lr * math.exp(math.log(self.final_lr_scale) * (self._step_count - self.warmup_updates) / self.decay_updates)
            else:
                base_lr = base_lr * self.final_lr_scale
            base_lrs_out.append(base_lr)
        return base_lrs_out


class W2V2CTCModel(LightningModule):
    """Wav2Vec2 finetune model."""

    def __init__(self, labels, state_dict={}):
        super().__init__()

        # instantiate torchaudio model build
        self.model = torchaudio.models.wav2vec2_base(
            encoder_projection_dropout= 0.1,
            encoder_attention_dropout = 0.1,
            encoder_ff_interm_dropout = 0.1,
            encoder_dropout = 0.1,
            encoder_layer_drop = 0.1
        )
        self.labels = labels
        # set output dimension size (length of vocabulary)
        ctc_layer_dimension = len(self.labels)
        # instantiate linear transform layer. Takes output from encoder.
        self.aux = torch.nn.Linear(768, ctc_layer_dimension)
        # load pretrained model
        self.model.load_state_dict(state_dict, strict=False)
        # freeze feature extraction layers
        for p in self.model.feature_extractor.parameters():
            p.requires_grad = False
        self.loss_fn = torch.nn.CTCLoss(zero_infinity=True)
        self.optimizer = torch.optim.AdamW(
            list(self.aux.parameters()) + list(self.model.parameters()),
            lr=1e-4,
            weight_decay=0.005,
        )
        self.lr_scheduler = BiStageLRScheduler(self.optimizer, 1000, 19000)
        # configure lr scheduler to be called at each step
        self.lr_scheduler_config = {
            "scheduler": self.lr_scheduler,
            "interval": "step"
        }
        # disable automatic lightning optimization (necessary for customizing train step)
        self.automatic_optimization = False
        # used for mixed precision training in order to reduce gradient underflow
        self.scaler = torch.cuda.amp.GradScaler()
        # number of step to freezing encoder parameters.
        self.freeze_encoder_updates = 1000

    def forward(self, x, lengths=None):
        """Forward pass through model, used during inference."""

        x, lengths = self._get_features(x, lengths)
        x = self._encode_features(x, lengths)
        x = self.aux(x)
        return x

    def _get_features(self, wavs, wav_lens):
        """Extracts feature vectors from raw audio Tensor."""

        with torch.no_grad():
            wavs_feats, wav_feats_lens = self.model.feature_extractor(wavs, wav_lens)
            return wavs_feats, wav_feats_lens

    def _encode_features(self, wavs, wav_lens):
        """Converts the audio features into the sequence of probability
        distribution (in negative log-likelihood) over labels."""

        x, _ = self.model.encoder._preprocess(wavs, wav_lens)
        x = self.model.encoder.transformer(x, attention_mask=None)
        return x

    def _log_error_rates(self, refs, log_probs):
        """Estimates and logs char and word error on validation data"""

        targets = [get_transcription(ref, self.labels) for ref in refs]
        with torch.inference_mode():
            preds = [viterbi_decode(log_prob, self.labels) for log_prob in log_probs]
        cer, wer = get_error_rates(preds, targets)
        self.log(f"val_cer", cer, on_step=True, on_epoch=True, prog_bar=True)
        self.log(f"val_wer", wer, on_step=True, on_epoch=True, prog_bar=True)

    def _loss_fn(self, batch, batch_idx, step_type=""):
        """Applies per batch feature extraction and loss estimation."""

        if batch is not None:
            waveforms, waveform_lengths, labels, label_lengths = batch
            x, out_len = self._get_features(waveforms, waveform_lengths)
            if self.global_step <= self.freeze_encoder_updates:
                with torch.no_grad():
                    x = self._encode_features(x, out_len)
            else:
                x = self._encode_features(x, out_len)

            aux_logits = self.aux(x)
            log_probs = F.log_softmax(aux_logits, dim=-1)
            log_probs_trans = log_probs.transpose(0, 1)
            loss = self.loss_fn(log_probs_trans, labels, out_len, label_lengths)
            self.log(f"{step_type}_loss", loss.item()/waveforms.size(0), on_step=True, on_epoch=True, prog_bar=True)

            if step_type == 'val':
                self._log_error_rates(labels, log_probs)
            return loss
        return batch

    def configure_optimizers(self):
        """
        Configures optimizers.
        Return two lists: first is the optimizer's and the second is the scheduler's.
        """

        return [self.optimizer],  [self.lr_scheduler_config]

    def training_step(self, batch, batch_idx):
        """Manual optimization training step"""

        opt = self.optimizers()
        # resets gradients before parameter updates
        opt.zero_grad()
        # apply mixed precision training
        try:
            with torch.cuda.amp.autocast(enabled=True):
                loss = self._loss_fn(batch, batch_idx, "train")
        except:
            with torch.cpu.amp.autocast(enabled=True):
                loss = self._loss_fn(batch, batch_idx, "train")

        # scales loss to prevent vanishing gradients
        loss = self.scaler.scale(loss)
        self.manual_backward(loss)
        # Unscales the gradients of optimizer's assigned params in-place
        # Calling before clipping enables you to clip unscaled gradients as usual
        self.scaler.unscale_(opt)
        # gradient clipping manipulates a set of gradients such that their global norm is <= threshold value
        self.clip_gradients(opt, gradient_clip_val=1.0, gradient_clip_algorithm="norm")

        # Unscales the gradients of the optimizer's assigned params.
        self.scaler.step(opt)
        # stepwise scheduler call
        sch = self.lr_schedulers()
        sch.step()
        # Updates the scale for next iteration.
        self.scaler.update()
        return loss

    def validation_step(self, batch, batch_idx):
        """Estimate validation loss"""

        return self._loss_fn(batch, batch_idx, "val")

## Train

In [ ]:
# download pretrain model
model = BUNDLE_PRE.get_model()
# set path for model
model_path = 'w2v2_base_pretrained.pt'
# save wav2vec2 pretrain model to path
torch.save(model, model_path)
checkpoint = model_path
model_pretrain = torch.load(checkpoint, map_location=torch.device("cpu"))
STATE_DICT = copy.deepcopy(model_pretrain.state_dict())


def run_train():
    """Confgure and run training"""

    seed_everything(1337)
    checkpoint_dir = "/content/checkpoints"
    checkpoint = ModelCheckpoint(
        dirpath=checkpoint_dir,
        monitor="val_wer",
        mode="min",
        save_top_k=1,
        save_weights_only=True,
        verbose=True,
    )
    early_stopping = EarlyStopping(monitor="val_wer", min_delta=0.00, patience=3, verbose=False, mode="min")
    trainer = Trainer(
        max_steps=20000,
        callbacks=[checkpoint, early_stopping],
        reload_dataloaders_every_n_epochs=1,
        accelerator="auto",
        val_check_interval=1000,
        check_val_every_n_epoch=None
    )
    data_module = DataModule(data, LABELS, SAMPLE_RATE, batch_size=4)
    model = W2V2CTCModel(LABELS, STATE_DICT)
    trainer.fit(model, data_module)

In [ ]:
# run_train()

## Decode

In [ ]:
class Infer:
    """Class to predict text given speech input"""

    def __init__(self, checkpoint, labels, sample_rate):
        """Args:
            checkpoint (str): Trained model checpoint path.
            labels (:obj: list, optional): List of output labels (vocabulary).
            sample_rate (int, optional): Integer value sample rate of audio model was trained with.
        """

        self.device = torch.device("cpu")
        self.labels = labels
        self.model = self.load_checkpoint(checkpoint)

        self.tokenizer = InputProcessor(text=True, inference=True)
        self.extractor = InputProcessor(audio=True, inference=True, sample_rate=sample_rate)

    def load_checkpoint(self, checkpoint):
        """Load model checkpoint to perform inference."""

        model = W2V2CTCModel(self.labels)
        checkpoint = torch.load(checkpoint, map_location="cpu")
        state_dict = checkpoint["state_dict"]
        model.load_state_dict(state_dict)
        model.eval().to(self.device)
        return model

    def run_inference(self, waveform):
        """Run inference on input"""

        waveform = self.extractor(waveform)
        with torch.inference_mode():
            emission = self.model(waveform.to(self.device))
            emission = F.log_softmax(emission, dim=-1)
            hypothesis = viterbi_decode(emission, self.labels)
        return hypothesis

    def run_interactive(self):
        """"""

        gr.Interface(
            fn=self.run_inference,
            inputs=gr.Audio(source="microphone", type="filepath"),
            outputs="text").launch()

    def process_input(self, input_data, evaluate=False):
        """Preprocess input and either evaluate or generate text"""

        if evaluate:
            targets = [self.tokenizer(pair[1]).lower() for pair in input_data]
            preds = [self.run_inference(pair[0]) for pair in input_data]
            cer, wer = get_error_rates(preds, targets)
            for i, target in enumerate(targets[:5]):
                print(f"Reference: {target.lower()}\nHypothesis: {preds[i]}\n---------------------------------")
            print(f"CER: {cer}, WER: {wer}")
        else:
            self.run_interactive()

In [ ]:
checkpoint = '/content/fintuned_wav2vec2.ckpt'
DATASET_DEV = "/content/data_dev"
input_data = data = get_transcript_audio_pair_list(DATASET_DEV)
infer = Infer(checkpoint, LABELS, SAMPLE_RATE)
infer.process_input(input_data, evaluate=True)

Reference: explain bias variance trade off
Hypothesis: explain bias and variance dato
---------------------------------
Reference: what is understood by the term data science
Hypothesis: what is understod by the term data science
---------------------------------
Reference: explain the differences between supervised and unsupervised learning
Hypothesis: explain the differences between supervised and unsupervised learning
---------------------------------
Reference: how to make a decision tree
Hypothesis: how to make a decision tree
---------------------------------
Reference: can you cite an example where a false positive is more important than a false negative
Hypothesis: can you cite an example wher a false positive is more important than a false negative
---------------------------------
CER: 0.05098039284348488, WER: 0.11627907305955887
